In [3]:
import importlib, os, gc, sys
import SXBQ as sx

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.optimize import fsolve, fmin
from scipy.signal import convolve as conv
import gsw
import cmocean.cm as cmo
from datetime import datetime as dt
import matplotlib.dates as mdates
import math

from matplotlib.gridspec import GridSpec

import glidertools as gt
import matplotlib.colors as colors
import matplotlib.cm as cm

import seaborn as sns
sns.set(    rc={
         'axes.axisbelow': False,
         'axes.edgecolor': 'Black',
         'axes.facecolor': 'white',
         'axes.grid': False,
         'axes.labelcolor': 'dimgray',
            
         'axes.spines.right': True,
         'axes.spines.top': True,
         'figure.facecolor': 'white',
         'lines.solid_capstyle': 'round',
         'patch.edgecolor': 'k',
         'patch.force_edgecolor': True,
         'text.color': 'dimgray',
         'xtick.bottom': True,
         'xtick.color': 'dimgray',
         'xtick.direction': 'out',
         'xtick.top': False,
         'ytick.color': 'dimgray',
         'ytick.direction': 'out',
         'ytick.left': True,
         'ytick.right': False},
         font_scale=1.6)

import warnings
warnings.filterwarnings('ignore')

def interp(x,y,xi):
    _gg = np.isfinite(x+y)
    return interp1d(x[_gg], y[_gg], bounds_error=False, fill_value=np.NaN)(xi)

## Import glider timeseries

In [4]:
## SG 2015-16  (Data Access:)
ds_79= xr.open_dataset('/home/jupyter-estelfont/common/fridge/2015_16_OMAN/SG579_timeseries.nc')
ds_10= xr.open_dataset('/home/jupyter-estelfont/common/fridge/2015_16_OMAN/SG510_timeseries.nc')
ds_02= xr.open_dataset('/home/jupyter-estelfont/common/fridge/2015_16_OMAN/SG502_timeseries.nc')

# # SeaExplorer 2021-22  (Data Access:)
ds= xr.open_dataset('/home/jupyter-estelfont/common/kitchensink/for_Bastien_oman/SEA057_glider.nc')
dadcp= xr.open_dataset('/home/jupyter-estelfont/common/kitchensink/for_Bastien_oman/SEA057_AD2CP.nc')

## SeaExplorer and SeaGlider basic processing and flagging

In [5]:
ds.abs_salinity[ds.flag==True]=np.nan
ds.cons_temp[ds.flag==True]=np.nan
ds.oxygen_concentration[ds.flag==True]=np.nan
ds.potential_density[ds.flag==True]=np.nan
ds['pressure']=ds.pressure.ffill(dim='time').bfill(dim='time')

def _process(data):
    data['spice']=('time', gsw.spiciness0(data.abs_salinity, data.cons_temp).values   )
    o2_sol = gsw.O2sol(data['abs_salinity'].values,data['cons_temp'].values,data['pressure'].values,data['longitude'].values,data['latitude'].values)
    ds['aou'] =('time', (o2_sol - data['oxygen_concentration']).values)
    return data

ds = _process(ds)

dadcp['flag']=('time', interp(ds.time.values.astype(float), ds.flag.values, dadcp.time.values))    
dadcp['speed']=('time', np.sqrt(dadcp['V_refE']**2+dadcp['V_refN']**2).values)
dadcp['potential_density']=('time', interp(ds.time.values.astype(float), ds.potential_density.values, dadcp.time.values))    
dadcp['V_along']=dadcp['V_refE']*np.cos(np.deg2rad(25))-dadcp['V_refN']*np.sin(np.deg2rad(25))
dadcp['V_across']=dadcp['V_refE']*np.sin(np.deg2rad(25))+dadcp['V_refN']*np.cos(np.deg2rad(25))
dadcp['abs_salinity']=('time', interp(ds.time.values.astype(float), ds.abs_salinity.values, dadcp.time.values))    
dadcp['cons_temp']=('time', interp(ds.time.values.astype(float), ds.cons_temp.values, dadcp.time.values))    

In [6]:
def _SG_process(data):    
    from datetime import datetime

    def date2float(d, epoch=pd.to_datetime(0)):
        return (d - epoch).dt.total_seconds()

    offset = datetime(1970, 1, 1).toordinal() + 366  #719529
    datatime=pd.to_datetime(data.time.values-offset, unit='D')
    # Produce a time array in seconds, as we always need it, for shorthand:
    time = date2float(pd.DataFrame(datatime)[0])
    time = time-time[0]
    data['timestamp'] = ('time', datatime)
    data['date_float'] = ('time', date2float(pd.DataFrame(datatime)[0]))
    data['time']=data.timestamp
    
    data=data.rename_vars({'lon':'longitude',  'lat':'latitude' , 'temp':'temperature', 'oxygen':'oxygen_concentration'})
    data['abs_salinty'] = ('time', gsw.SA_from_SP((data['salinity']*data['flag']).values,data['pressure'].values,data['longitude'].values,data['latitude'].values))
    data['cons_temp'] = ('time',gsw.CT_from_t(data['abs_salinty'].values,(data['temperature']*data['flag']).values,data['pressure'].values))
    data['potential_density'] = ('time',gsw.sigma0(data['abs_salinty'].values,data['cons_temp'].values))
    # data['soundspeed'] = ('time',gsw.sound_speed(data['sa'].values,data['ct'].values,data['pressure'].values))
    data['spice']=('time', gsw.spiciness0(data.abs_salinty, data.cons_temp).values   )

    o2_sol = gsw.O2sol(data['abs_salinty'],data['cons_temp'],data['pressure'],data['longitude'],data['latitude'])
    # o2_sat =  data['AROD_FT_DO'] / gsw.O2sol( data['sa']*0, data['AROD_FT_TEMP'], data['pressure']*0,data['longitude'],data['latitude'])
    # data['o2'] = ('time',(o2_sat * o2_sol).values)
    # data['o2_sat'] =('time', (o2_sat * 100).values)
    data['aou'] =('time', (o2_sol - data['oxygen_concentration']).values)
    return data

ds_79 = _SG_process(ds_79)
ds_10 = _SG_process(ds_10)
ds_02 = _SG_process(ds_02)

ds_10['profile_num']=ds_10.profile_num.ffill('time')
ds_79['profile_num']=ds_79.profile_num.ffill('time')
ds_02['profile_num']=ds_02.profile_num.ffill('time')

## MLD and mode water calculation and masking

In [7]:
def mld_ds(df):
    mld_=gt.physics.mixed_layer_depth(df.profile_num.values, df.pressure.values, df.potential_density.values, thresh=0.125, ref_depth=2, return_as_mask=False)
    # ds['mld']=('profile_num', mld_.to_numpy())

    df=df.set_coords('profile_num')
    df['mld']=('profile_num', mld_)
    df['mld']=df['mld'].interpolate_na('profile_num')
    
    # ds['mld+25']=('profile_num', mld_+25)
    df['mixed_layer_depth']=('time', interp(np.unique(df.profile_num.values), 
                                            pd.DataFrame(df.mld.values).rolling(5, center=True).mean()[0].to_numpy(), df.profile_num.values))

    msk_mld= (df.pressure.values > (df['mixed_layer_depth'].values+25)).astype(float)
    msk_mld[msk_mld==0]=np.nan
    df['mld_mask']=('time',msk_mld)
    return df


ds=mld_ds(ds)
ds_02=mld_ds(ds_02)
ds_10=mld_ds(ds_10)
ds_79=mld_ds(ds_79)

ds_10.mld[68:220]=np.nan
ds_10['mld']=ds_10['mld'].interpolate_na('profile_num')
ds_10['mixed_layer_depth']=('time', interp(np.unique(ds_10.profile_num.values), 
                                            pd.DataFrame(ds_10.mld.values).rolling(5, center=True).mean()[0].to_numpy(), ds_10.profile_num.values))

msk_mld= (ds_10.pressure.values > (ds_10['mixed_layer_depth'].values+25)).astype(float)
msk_mld[msk_mld==0]=np.nan
ds_10['mld_mask']=('time',msk_mld)

In [8]:
msk_wwl= (1500 < ds.profile_num) & (ds.profile_num.values <2000) & (ds.potential_density > 25.3)
msk_wwl[ds.profile_num<1500]=True
msk_wwl[ds.profile_num>2000]=True
msk_wwl=msk_wwl.values.astype(float)
msk_wwl[msk_wwl==0]=np.nan
ds['wwl_mask']=('time',msk_wwl)

msk_wwl= (ds_02.potential_density > 25.3)
msk_wwl=msk_wwl.values.astype(float)
msk_wwl[msk_wwl==0]=np.nan
ds_02['wwl_mask']=('time',msk_wwl)

msk_wwl= (ds_10.potential_density > 25.3)
msk_wwl=msk_wwl.values.astype(float)
msk_wwl[msk_wwl==0]=np.nan
ds_10['wwl_mask']=('time',msk_wwl)

msk_wwl= (ds_79.potential_density > 25.3)
msk_wwl=msk_wwl.values.astype(float)
msk_wwl[msk_wwl==0]=np.nan
ds_79['wwl_mask']=('time',msk_wwl)

## Transect numbers

In [9]:
arr_10=[0, 294,  367,  419,  497,  547,  595  ,630, 655,  722, 
          798,  848,  890,  932,  978,
       1027, 1075, 1117, 1178,  1241,1293, 1356,
        1404, 1457, 1500, 1548]

arr_79=[   0,       75,   155,   235,  298,  364,  413, 460,  
        516,   560,  633,  668,  741,  780,    838,  882,  928, 
        975,  1020, 1056, 1125,  1174, 1227, 1260, 
        1308]

arr_02=[  0, 33, 90, 157, 215, 258, 372]
arr_2022=[   0,   39,  109,  149,  157,  358,  375,  436,  493,  502,  543,
         581,  613,  663,  709,  751,  795,  841,  861,  895,  929,  981,
        1017, 1091, 1131, 1206, 1215, 1233, 1277, 1315, 1367, 1387, 1433,
        1485, 1581, 1633, 1649, 1665, 1707, 1747, 1821, 1877, 1901, 2036,
        2093, 2133, 2175, 2247, 2297, 2339, 2383, 2417, 2443, 2483, 2529,
        2573, 2703, 2747, 2809, 2861, 2903, 2953, 3025, 3115, 3167, 3233,
        3289, 3337, 3380, 3429]

up_down_10=[0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1]
up_down_79=[0,2,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1]
up_down_02=[2,0, 1,0,1,0,1]
up_down_2022=[0,1,0,1,2,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,2, 0, 1, 0,1,0,1,0,1,0, 2, 1, 0,1,0,1,0,1,0,1, 0,1,0,1,0,1,0,1]

def trans_num(transectNum, ds, up_down):
    arr2=np.array([])
    arr2ud=np.array([])

    NumT=np.arange(1,len(transectNum)+1, 1)
    for m in tqdm(range(len(transectNum))):
        if m<(len(up_down)-1):
            arr=np.ones(len(ds.profile_num[(ds.profile_num>=transectNum[m]) & (ds.profile_num<transectNum[m+1])]))*NumT[m]
            arr2=np.concatenate([arr2, arr])
            arrud=np.ones(len(ds.profile_num[(ds.profile_num>=transectNum[m]) & (ds.profile_num<transectNum[m+1])]))*up_down[m]
            arr2ud=np.concatenate([arr2ud, arrud])
        else:
            arr=np.ones(len(ds.profile_num[(ds.profile_num>=transectNum[m])]))*NumT[m]
            arr2=np.concatenate([arr2, arr])
            arrud=np.ones(len(ds.profile_num[(ds.profile_num>=transectNum[m])]))*up_down[m]
            arr2ud=np.concatenate([arr2ud, arrud])
            
    ds["transect_num"]=('time',  arr2)
    ds["transect_horientation"]=('time',  arr2ud)  ## 0 up, 1 down, 2 VM
    return ds

ds_10=trans_num(arr_10, ds_10, up_down_10)
ds_79=trans_num(arr_79, ds_79, up_down_79)
ds_02=trans_num(arr_02, ds_02, up_down_02)

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

## Water mass fraction

In [10]:
def watermass_fraction(theta, sa):
    ASW_sa, ASW_theta=gsw.SA_from_SP(34.7, 0, 58,24),  gsw.CT_from_t(gsw.SA_from_SP(34.7, 0, 58,24), 1,0)#35, 2.9
    PGW_sa, PGW_theta= gsw.SA_from_SP( 40.85, 0, 58,24),  gsw.CT_from_t(gsw.SA_from_SP( 40.85, 0, 58,24), 29.8,0)
    mW_sa, mW_theta= gsw.SA_from_SP(36.2, 0, 58,24),  gsw.CT_from_t(gsw.SA_from_SP(36.2, 0, 58,24), 23.5 ,0) #35.57, 15.8 #36, 25.5       #36, 24

    term1=theta-mW_theta-((sa-mW_sa)*(ASW_theta-mW_theta)/(ASW_sa-mW_sa))
    term2=PGW_theta-mW_theta-((PGW_sa-mW_sa)*(ASW_theta-mW_theta)/(ASW_sa-mW_sa))
    fract=term1/term2
    return fract

ds['pgw_frac']=('time', watermass_fraction(ds.cons_temp.values, ds.abs_salinity.values))
ds_79['pgw_frac']=('time', watermass_fraction(ds_79.cons_temp.values, ds_79.abs_salinity.values))
ds_10['pgw_frac']=('time', watermass_fraction(ds_10.cons_temp.values, ds_10.abs_salinity.values))
ds_02['pgw_frac']=('time', watermass_fraction(ds_02.cons_temp.values, ds_02.abs_salinity.values))

In [11]:
def watermass_fraction_icw(theta, sa):
    ASW_sa, ASW_theta=gsw.SA_from_SP(34.7, 0, 58,24),  gsw.CT_from_t(gsw.SA_from_SP(34.7, 0, 58,24), 1,0)#35, 2.9
    PGW_sa, PGW_theta= gsw.SA_from_SP( 40.85, 0, 58,24),  gsw.CT_from_t(gsw.SA_from_SP( 40.85, 0, 58,24), 29.8,0)
    mW_sa, mW_theta= gsw.SA_from_SP(36.2, 0, 58,24),  gsw.CT_from_t(gsw.SA_from_SP(36.2, 0, 58,24), 23.5 ,0) #35.57, 15.8 #36, 25.5       #36, 24
    term1=theta-ASW_theta-((sa-ASW_sa)*(PGW_theta-ASW_theta)/(PGW_sa-ASW_sa))
    term2=mW_theta-ASW_theta-((mW_sa-ASW_sa)*(PGW_theta-ASW_theta)/(PGW_sa-ASW_sa))
    fract=term1/term2
    return fract

ds['icw_frac']=('time', watermass_fraction_icw(ds.cons_temp.values, ds.abs_salinity.values))
ds_79['icw_frac']=('time', watermass_fraction_icw(ds_79.cons_temp.values, ds_79.abs_salinity.values))
ds_10['icw_frac']=('time', watermass_fraction_icw(ds_10.cons_temp.values, ds_10.abs_salinity.values))
ds_02['icw_frac']=('time', watermass_fraction_icw(ds_02.cons_temp.values, ds_02.abs_salinity.values))

In [12]:
def watermass_fraction_dasw(theta, sa):
    ASW_sa, ASW_theta=gsw.SA_from_SP(34.7, 0, 58,24),  gsw.CT_from_t(gsw.SA_from_SP(34.7, 0, 58,24), 1,0)#35, 2.9
    PGW_sa, PGW_theta= gsw.SA_from_SP( 40.85, 0, 58,24),  gsw.CT_from_t(gsw.SA_from_SP( 40.85, 0, 58,24), 29.8,0)
    mW_sa, mW_theta= gsw.SA_from_SP(36.2, 0, 58,24),  gsw.CT_from_t(gsw.SA_from_SP(36.2, 0, 58,24), 23.5 ,0) #35.57, 15.8 #36, 25.5       #36, 24

    
    term1=theta-PGW_theta-((sa-PGW_sa)*(mW_theta-PGW_theta)/(mW_sa-PGW_sa))
    term2=ASW_theta-PGW_theta-((ASW_sa-PGW_sa)*(mW_theta-PGW_theta)/(mW_sa-PGW_sa))
    fract=term1/term2
    return fract
ds['asw_frac']=('time', watermass_fraction_dasw(ds.cons_temp.values, ds.abs_salinity.values))
ds_79['asw_frac']=('time', watermass_fraction_dasw(ds_79.cons_temp.values, ds_79.abs_salinity.values))
ds_10['asw_frac']=('time', watermass_fraction_dasw(ds_10.cons_temp.values, ds_10.abs_salinity.values))
ds_02['asw_frac']=('time', watermass_fraction_dasw(ds_02.cons_temp.values, ds_02.abs_salinity.values))


for dff in [ds, ds_79, ds_10, ds_02]:
    dff.pgw_frac[dff.asw_frac<0]=np.nan
    dff.pgw_frac[dff.asw_frac>1]=np.nan
    dff.pgw_frac[dff.icw_frac<0]=np.nan
    dff.pgw_frac[dff.icw_frac>1]=np.nan
    dff.pgw_frac[dff.pgw_frac>1]=np.nan
    dff.pgw_frac[dff.pgw_frac>1]=np.nan

## Detide adcp data

In [13]:
ds_tides=xr.open_dataset('/home/jupyter-estelfont/PGW_paper/tidal_consituent/tides_202122.nc')

yaxis=np.arange(0,1000,1)
xaxis=np.unique(ds.profile_num)

V_refE,_,_=sx.grid2d(dadcp.profile_num, dadcp.pressure, dadcp.V_refE, xi=xaxis, yi=yaxis)
V_refN,_,_=sx.grid2d(dadcp.profile_num, dadcp.pressure, dadcp.V_refN, xi=xaxis, yi=yaxis)

dadcp['V_refE_detided']=('time', (V_refE-ds_tides.E_tide.values).T.flatten())
dadcp['V_refN_detided']=('time', (V_refN-ds_tides.N_tide.values).T.flatten())
dadcp['V_along_detided']=dadcp['V_refE_detided']*np.cos(np.deg2rad(25))-dadcp['V_refN_detided']*np.sin(np.deg2rad(25))
dadcp['V_across_detided']=dadcp['V_refN_detided']*np.cos(np.deg2rad(25))+dadcp['V_refE_detided']*np.sin(np.deg2rad(25))

### save datasets!

In [21]:
ds_79.to_netcdf('/home/jupyter-estelfont/datasets/oman_201516/final/SG579.nc')
ds_02.to_netcdf('/home/jupyter-estelfont/datasets/oman_201516/final/SG502.nc')
ds_10.to_netcdf('/home/jupyter-estelfont/datasets/oman_201516/final/SG510.nc')

ds.to_netcdf('/home/jupyter-estelfont/datasets/oman_202122/final/SEA057.nc')
dadcp.to_netcdf('/home/jupyter-estelfont/datasets/oman_202122/final/SEA057_adcp.nc')

# Grid dataset per transects

In [16]:
ds_79=xr.open_dataset('/home/jupyter-estelfont/datasets/oman_201516/final/SG579.nc')
ds_02=xr.open_dataset('/home/jupyter-estelfont/datasets/oman_201516/final/SG502.nc')
ds_10=xr.open_dataset('/home/jupyter-estelfont/datasets/oman_201516/final/SG510.nc')
ds=xr.open_dataset('/home/jupyter-estelfont/datasets/oman_202122/final/SEA057.nc')
dadcp=xr.open_dataset('/home/jupyter-estelfont/datasets/oman_202122/final/SEA057_adcp.nc')

## Grid in pressure

In [26]:
# gridding functions

daxis=np.arange(-10,80,1)
yaxis=np.arange(0,1000,2)
def sect_(data,  var, yaxis_, N):
    var_g, x, y=  sx.grid2d(
        data.distance[data.transect_num==N].values,
        data.pressure[data.transect_num==N].values, 
        data[var][data.transect_num==N].values, 
        xi=daxis, yi=yaxis, fn='mean')
    return  var_g


def sect_msk(data,  var, yaxis_, N):
    var_g, x, y=  sx.grid2d(
        data.distance[data.transect_num==N].values,
        data.pressure[data.transect_num==N].values, 
        (data[var]*data['mld_mask']*data['wwl_mask'])[data.transect_num==N].values,   #*data['wwl_mask']
        xi=daxis, yi=yaxis, fn='mean')
    return  var_g

def rol_(lol):
    lol=pd.DataFrame(lol).interpolate(axis=0, limit=5, limit_direction='both', limit_area='inside').interpolate(axis=1, limit=4, limit_direction='both', limit_area='inside').rolling(3, axis=1, center=True).mean().to_numpy()
    return lol

# variables to grid
variables_to_be_gridded=['abs_salinity', 'cons_temp', 'oxygen_concentration', 'pgw_frac', 'potential_density']

### SG502, 510, 579

In [30]:
ds_sections_02 = xr.Dataset(
    coords=dict(
        trans_num=(["trans_num"], np.unique(ds_02.transect_num)),
        y=(["y"], yaxis),
        x=(["x"], daxis),      
    ),
    attrs={},
)

for i in variables_to_be_gridded:
    PGW_sects=[rol_(sect_(ds_02, i, yaxis, N)) for N in tqdm(np.unique(ds_02.transect_num))]
    ds_sections_02[i]=(('trans_num',  'y', 'x' ), np.array(PGW_sects))
    
#####
ds_sections_10 = xr.Dataset(
    coords=dict(
        trans_num=(["trans_num"], np.unique(ds_10.transect_num)),
        y=(["y"], yaxis),
        x=(["x"], daxis),      
    ),
    attrs={},
)

for i in variables_to_be_gridded:
    PGW_sects=[rol_(sect_(ds_10, i, yaxis, N)) for N in tqdm(np.unique(ds_10.transect_num))]
    ds_sections_10[i]=(('trans_num',  'y', 'x' ), np.array(PGW_sects))
    
##### 
ds_sections_79 = xr.Dataset(
    coords=dict(
        trans_num=(["trans_num"], np.unique(ds_79.transect_num)),
        y=(["y"], yaxis),
        x=(["x"], daxis),      
    ),
    attrs={},
)

for i in variables_to_be_gridded:
    PGW_sects=[rol_(sect_(ds_79, i, yaxis, N)) for N in tqdm(np.unique(ds_79.transect_num))]
    ds_sections_79[i]=(('trans_num',  'y', 'x' ), np.array(PGW_sects))
    

## Geostrophic velocities    
ds_t= xr.open_dataset('/home/jupyter-estelfont/common/fridge/2015_16_OMAN/transects.nc')
ds_t['y']=np.arange(0,1004,4)

sg_79_=np.array([ 0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25])
Geo_velo=[-ds_t.geostrophy[i].interp(x=daxis, y=yaxis).values.T for i in sg_79_]
ds_sections_79['geostrophy']=(('trans_num',  'y', 'x' ), np.array(Geo_velo))

sg_02_=np.array([28, 29, 30, 31, 32, 33, 34])
Geo_velo=[-ds_t.geostrophy[i].interp(x=daxis, y=yaxis).values.T for i in sg_02_]
ds_sections_02['geostrophy']=(('trans_num',  'y', 'x' ), np.array(Geo_velo))

sg_10_=np.array([60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85])
Geo_velo=[-ds_t.geostrophy[i].interp(x=daxis, y=yaxis).values.T for i in sg_10_]
ds_sections_10['geostrophy']=(('trans_num',  'y', 'x' ), np.array(Geo_velo))

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

### SEA057

In [32]:
ds_sections_57 = xr.Dataset(
    coords=dict(
        trans_num=(["trans_num"], np.unique(ds.transect_num)),
        y=(["y"], yaxis),
        x=(["x"], daxis),      
    ),
    attrs={},
)

for i in variables_to_be_gridded:
    PGW_sects=[rol_(sect_(ds, i, yaxis, N)) for N in tqdm(np.unique(ds.transect_num))]
    ds_sections_57[i]=(('trans_num',  'y', 'x' ), np.array(PGW_sects))
    
for i in ['V_along_detided', 'V_across_detided']:
    PGW_sects=[rol_(sect_(dadcp, i, yaxis, N)) for N in tqdm(np.unique(dadcp.transect_num))]
    ds_sections_57[i]=(('trans_num',  'y', 'x' ), np.array(PGW_sects))

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

### Average timestamp per transect

In [33]:
def trans_taxis(fr):
    transmean, tra_n, _=sx.grid2d(fr.transect_num.values, fr.pressure.values, fr.time.values, xi=np.arange(0.5,np.nanmax(fr.transect_num.values)+1.5,1), yi=1000, fn='mean')
    taxis_transect_mean=pd.to_datetime(transmean[0])[:-1]
    return taxis_transect_mean

taxis_trans_02=trans_taxis(ds_02)
taxis_trans_79=trans_taxis(ds_79)
taxis_trans_10=trans_taxis(ds_10)
taxis_trans_57=trans_taxis(ds)

ds_sections_02['time']=(('trans_num'), taxis_trans_02)
ds_sections_79['time']=(('trans_num'), taxis_trans_79)
ds_sections_10['time']=(('trans_num'), taxis_trans_10)
ds_sections_57['time']=(('trans_num'), taxis_trans_57)

## add a time grid for sea057 -- timestamp per gridded profile
PGW_sects=[rol_(sect_(ds, 'time', yaxis, N)) for N in tqdm(np.unique(ds.transect_num))]
ds_sections_57['time_grid']=(('trans_num',  'y', 'x' ), np.array(PGW_sects))

##### Save gridded datasets!

In [41]:
ds_sections_02.to_netcdf('/home/jupyter-estelfont/datasets/oman_201516/final/gridded_SG502.nc')
ds_sections_79.to_netcdf('/home/jupyter-estelfont/datasets/oman_201516/final/gridded_SG579.nc')
ds_sections_10.to_netcdf('/home/jupyter-estelfont/datasets/oman_201516/final/gridded_SG510.nc')
ds_sections_57.to_netcdf('/home/jupyter-estelfont/datasets/oman_202122/final/gridded_SEA057_1.nc')

## Grid in potential density

In [43]:
sigmaaxis=np.arange(20,28,0.005)

def sect_sigma(data,  var, yaxis_, N):
    var_g, x, y=  sx.grid2d(
        data.distance[data.transect_num==N].values,
        data.potential_density[data.transect_num==N].values, 
        data[var][data.transect_num==N].values, 
        xi=np.arange(-10,80,1), yi=sigmaaxis, fn='mean')
    return  var_g

def sect_sigma_msk(data,  var, yaxis_, N):
    var_g, x, y=  sx.grid2d(
        data.distance[data.transect_num==N].values,
        data.potential_density[data.transect_num==N].values, 
        (data[var]*data['mld_mask']*data['wwl_mask'])[data.transect_num==N].values, 
        xi=np.arange(-10,80,1), yi=sigmaaxis, fn='mean')
    return  var_g

### SG502, 510, 579

In [44]:
ds_sections_02_SIGMA = xr.Dataset(
    coords=dict(
        trans_num=(["trans_num"], np.unique(ds_02.transect_num)),
        y=(["y"], sigmaaxis),
        x=(["x"], daxis),      
    ),
    attrs={},
)

for i in variables_to_be_gridded:
    PGW_sects=[rol_(sect_sigma(ds_02, i, sigmaaxis, N)) for N in tqdm(np.unique(ds_02.transect_num))]
    ds_sections_02_SIGMA[i]=(('trans_num',  'y', 'x' ), np.array(PGW_sects))
    
ds_sections_79_SIGMA = xr.Dataset(
    coords=dict(
        trans_num=(["trans_num"], np.unique(ds_79.transect_num)),
        y=(["y"], sigmaaxis),
        x=(["x"], daxis),      
    ),
    attrs={},
)

for i in variables_to_be_gridded:
    PGW_sects=[rol_(sect_sigma(ds_79, i, sigmaaxis, N)) for N in tqdm(np.unique(ds_79.transect_num))]
    ds_sections_79_SIGMA[i]=(('trans_num',  'y', 'x' ), np.array(PGW_sects))
    
ds_sections_10_SIGMA = xr.Dataset(
    coords=dict(
        trans_num=(["trans_num"], np.unique(ds_10.transect_num)),
        y=(["y"], sigmaaxis),
        x=(["x"], daxis),      
    ),
    attrs={},
)

for i in variables_to_be_gridded:
    PGW_sects=[rol_(sect_sigma(ds_10, i, sigmaaxis, N)) for N in tqdm(np.unique(ds_10.transect_num))]
    ds_sections_10_SIGMA[i]=(('trans_num',  'y', 'x' ), np.array(PGW_sects))

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

### SEA057

In [47]:
ds_sections_57_SIGMA = xr.Dataset(
    coords=dict(
        trans_num=(["trans_num"], np.unique(ds.transect_num)),
        y=(["y"], sigmaaxis),
        x=(["x"], daxis),      
    ),
    attrs={},
)

for i in variables_to_be_gridded:
    PGW_sects=[rol_(sect_sigma(ds, i, sigmaaxis, N)) for N in tqdm(np.unique(ds.transect_num))]
    ds_sections_57_SIGMA[i]=(('trans_num',  'y', 'x' ), np.array(PGW_sects))

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

##### Save gridded datasets!

In [48]:
ds_sections_02_SIGMA.to_netcdf('/home/jupyter-estelfont/datasets/oman_201516/final/gridded_SG502_sigma.nc')
ds_sections_79_SIGMA.to_netcdf('/home/jupyter-estelfont/datasets/oman_201516/final/gridded_SG579_sigma.nc')
ds_sections_10_SIGMA.to_netcdf('/home/jupyter-estelfont/datasets/oman_201516/final/gridded_SG510_sigma.nc')
ds_sections_57_SIGMA.to_netcdf('/home/jupyter-estelfont/datasets/oman_202122/final/gridded_SEA057_sigma.nc')